In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
indicator = 'positive'

In [ ]:
with open(f'time_series_{indicator}.pkl', 'rb') as f:
    data = pkl.load(f)

with open(f'indices_{indicator}.pkl', 'rb') as f:
    indices = pkl.load(f)

In [ ]:
texts = {}
for i in indices:
    with open(os.path.join('gpt_summary', f'{i}_{indicator}.txt'), 'r') as f:
        text = f.read()
        texts[i] = text

In [ ]:
data_size = data.shape[0]
window_size = 20
print(data_size, data.shape, window_size)

In [ ]:
data_size = len(indices)

num_train = int(data_size * 0.6)
num_test = int(data_size * 0.2)
num_vali = data_size - num_train - num_test

idx_train = np.arange(num_train)
idx_valid = np.arange(num_train, num_train + num_vali)
idx_test = np.arange(num_train + num_vali, num_train + num_vali + num_test)

## Prompt GPT 4

In [ ]:
API_KEY = ''

In [ ]:
if indicator == 'positive':
    system_prompt = f"Your job is to act as a professional healthcare forecaster. You will be given a healthcare summary of the past 20 weeks. Based on this information, your task is to predict whether the percentage of respiratory specimens testing positive for influenza will exceed the average threshold in the comming week."
elif indicator == 'mortality':
    system_prompt = f"Your job is to act as a professional healthcare forecaster. You will be given a healthcare summary of the past 20 weeks. Based on this information, your task is to predict whether the ratio of mortality from Influenza or Pneumonia to the total number of death will exceed its average in the comming week."

print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for _i in idx_test:
    i = indices[_i]
    
    today = data[i][0]

    if indicator == 'posiive':
        user_prompt = f"Your task is to predict whether the percentage of respiratory specimens testing positive for influenza will:\n"
        user_prompt += "(1) Exceed its average of 6.26%\n(2) Not exceed its average of 6.26%\n"
        user_prompt += "in the coming week. "
        user_prompt += f"The healthcare situation of the last {window_size} weeks is summarized as follows:\n\n"
        user_prompt += f"{texts[i]}\n\n"
        user_prompt += f"Analyze this summary and predict whether the percentage of respiratory specimens testing positive for influenza will exceed the average of 6.26% or not. "
        user_prompt += "Respond with either \'exceed\' or \'not exceed\'. Do not provide any other details."

    elif indicator == 'mortality':
        user_prompt = f"Your task is to predict whether the ratio of mortality from Influenza or Pneumonia to the total number of death will:\n"
        user_prompt += "(1) Exceed its average of 7.84%\n(2) Not exceed its average of 7.84%\n"
        user_prompt += "in the coming week. "
        user_prompt += f"The healthcare situation of the last {window_size} weeks is summarized as follows:\n\n"
        user_prompt += f"{texts[i]}\n\n"
        user_prompt += f"Analyze this summary and predict whether the ratio of mortality from Influenza or Pneumonia to the total number of death will exceed the average of 7.84% or not. "
        user_prompt += "Respond with either \'exceed\' or \'not exceed\'. Do not provide any other details."
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content

    with open(f'gpt_predict_text/{i}_{indicator}.txt', 'w') as f:
        f.write(text)
    #break